Modify the Bagging scratch code in our lecture such that:
- Calculate for oob evaluation for each bootstrapped dataset, and also the average score
- Change the code to "without replacement"
- Put everything into a class <code>Bagging</code>.  It should have at least two methods, <code>fit(X_train, y_train)</code>, and <code>predict(X_test)</code>
- Modify the code from above to randomize features.  Set the number of features to be used in each tree to be <code>sqrt(n)</code>, and then select a subset of features for each tree.  This can be easily done by setting our DecisionTreeClassifier <code>max_features</code> to 'sqrt'

In [1]:
# Calculate for oob evaluation for each bootstrapped dataset, and also the average score

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.3, shuffle=True, random_state=42)

In [2]:
from sklearn.tree import DecisionTreeClassifier
import random, math
from scipy import stats
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

class RandomForest:
    def __init__(self, B, bootstrap_ratio, with_no_replacement=True):
        self.B = B    
        self.bootstrap_ratio = bootstrap_ratio
        self.with_no_replacement = with_no_replacement
        self.tree_params = {'max_depth': 2, 'max_features': 'sqrt'}  #rmse return math.sqrt
        self.models = [DecisionTreeClassifier(**self.tree_params) for _ in range(B)]

    # 1. X_train, y_train
    def fit(self, X, y):  
        m, n = X.shape

        #sample size for each tree
        sample_size = int(self.bootstrap_ratio * len(X))

        xsamples = np.zeros((self.B, sample_size, n))
        ysamples = np.zeros((self.B, sample_size))

        # use list because length is not known           #<-- Out of bag : keep in list
        xsamples_oob = []  
        ysamples_oob = []
       
        #bootstrapping samples (subsam) for each model
        for i in range(self.B):
            oob_idx = []
            idxes = []       
            for j in range(sample_size):
                idx = random.randrange(m)             # with replacement
                if (self.with_no_replacement):        # with no replace : oob
                    while idx in idxes:                    # we have to keep track what the indexes're already used
                        idx = random.randrange(m)
                idxes.append(idx)           
                oob_idx.append(idx)
                
                xsamples[i, j, :] = X[idx]
                ysamples[i, j] = y[idx]
                #keep track of idx that i did not use for ith tree
                
            mask = np.zeros((m), dtype=bool)           ##keep b(can see) and oob
            #print(mask)
            mask[oob_idx] = True
            #print(mask[oob_idx])
            
            xsamples_oob.append(X[~mask])
            ysamples_oob.append(y[~mask])
        #print(xsamples_oob)
        #print(ysamples_oob)
              
     #fitting each estimator
        oob_score = 0
        print("====== Out of bag score for each tree ======")
        
        for i, model in enumerate(self.models):
            _X = xsamples[i]
            _y = ysamples[i]
            model.fit(_X, _y)
            
            _X_test = np.asarray(xsamples_oob[i])                    # calculating oob score for each bootstrapped dataset
            _y_test = np.asarray(ysamples_oob[i])
            #print(_X_test)
            #print(_y_test)
            
            yhat = model.predict(_X_test)
            oob_score += accuracy_score(_y_test, yhat)
            print(f"Tree {i}", accuracy_score(_y_test, yhat))
    
        self.avg_oob_score = oob_score / len(self.models)           # the average score
        print("====== Average out of bag score ======")
        print(self.avg_oob_score)
        
    # 2. X_test
    def predict(self, X): 
        #make prediction and return the probabilities
        predictions = np.zeros((self.B, X.shape[0]))
        for i, model in enumerate(self.models):
            yhat = model.predict(X)
            predictions[i, :] = yhat
        return stats.mode(predictions)[0][0]


In [3]:
# try md
model = RandomForest(B=5, bootstrap_ratio = 0.8)

model.fit(X_train, y_train)
yhat = model.predict(X_test)

print(classification_report(y_test, yhat))

====== Out of bag score for each tree ======
Tree 0 1.0
Tree 1 0.8571428571428571
Tree 2 0.7619047619047619
Tree 3 0.9523809523809523
Tree 4 0.9523809523809523
====== Average out of bag score ======
0.9047619047619048
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

